In [3]:
import pandas as pd
import requests
import json
import config
import io

import geopandas as gpd
from geopandas import GeoDataFrame, points_from_xy
from cartoframes import read_carto, to_carto
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer, basemaps
from cartoframes.data.clients import SQLClient

PROJ: proj_create_from_database: Cannot find proj.db


In [4]:
credentials = pd.read_csv('credentials.csv',header=0)
username = credentials.columns[0]
api_key = credentials.values[0][0]

set_default_credentials(username=username,api_key=api_key)

credentials = Credentials(username=username,api_key=api_key)

sql = SQLClient(credentials)

In [5]:
zipper = pd.read_csv('Miami Zip codes.csv')
zips = zipper['Zip Code'].tolist()
zips.sort()

In [6]:
df = pd.DataFrame()

## to meet Git/Carto requirements
## can only use 2011 and forward but
## can go as far back as 1999 tho
for y in range(1999, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    if y==2019:
        dataset = 'cbp'
    else:
        dataset = 'zbp'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    if y==2019 or y==2018 or y==2017:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_LABEL,PAYANN,NAICS2017,NAICS2017_LABEL'
    elif y==2016 or y==2015 or y==2014 or y==2013 or y==2012:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2012,NAICS2012_TTL'
    elif y==2011 or y==2010 or y==2009 or y==2008:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2007,NAICS_TTL'
    elif y==2007 or y==2006 or y==2005 or y==2004 or y==2003:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2002,NAICS_TTL'
    ## Query error with 1998
    ## Logged by Census but cannot run as of 9/18/21
    elif y==2002 or y==2001 or y==2000 or y==1999 or y==1998:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS1997,NAICS_TTL'
    
    loc = '&for=zipcode:'+','.join(map(str,zips))
    
    full_url = base_url + var + loc
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    cols = ['estab', 'emp', 'empszes', 'est_size', 'payroll', 'naics', 'ind', 'zip', 'year']
    
    for c in range(0,len(cols)):
        df1.rename(columns={df1.columns[c]:cols[c]}, inplace=True)
    ## 2016 & before includes all establishment sizes variable rows
    ## even if there were no establishments of that size, so will remove
    df1 = df1[df1['estab']!='0']

    df = df.append(df1,ignore_index=True)

In [14]:
## Short of 3 provides most uniformity; 4 becomes too much
df['naics_short'] = df['naics'].apply(str).str[:3]

In [15]:
## Limit to 2011-2019 to meet size restraints
df = df[df['year'] > 2010]

In [16]:
## Export to csv for easy reload
df.to_csv("ind.csv", sep=",", index=False)

In [8]:
## Push to Carto
## Can't push to Carto
## DB quota exceeded
# to_carto(df, 'ind', if_exists='replace')

In [3]:
df = pd.DataFrame()

for y in range(2011, 2012):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    if y==2019:
        dataset = 'cbp'
    else:
        dataset = 'zbp'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    if y==2019 or y==2018 or y==2017:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_LABEL,PAYANN,NAICS2017,NAICS2017_LABEL'
    elif y==2016 or y==2015 or y==2014 or y==2013 or y==2012:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2012,NAICS2012_TTL'
    elif y==2011 or y==2010 or y==2009 or y==2008:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2007,NAICS_TTL'
    elif y==2007 or y==2006 or y==2005 or y==2004 or y==2003:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2002,NAICS_TTL'
    ## Query error with 1998
    ## Logged by Census but cannot run as of 9/18/21
    elif y==2002 or y==2001 or y==2000 or y==1999 or y==1998:
        var = 'ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS1997,NAICS_TTL'
    
    loc = '&for=zipcode:'+','.join(map(str,zips))
    
    full_url = base_url + var + loc
    
    print(full_url)

https://api.census.gov/data/2011/zbp/?get=ESTAB,EMP,EMPSZES,EMPSZES_TTL,PAYANN,NAICS2007,NAICS_TTL&for=zipcode:33010,33012,33013,33014,33015,33016,33018,33030,33031,33032,33033,33034,33035,33039,33054,33055,33056,33101,33109,33122,33125,33126,33127,33128,33129,33130,33131,33132,33133,33134,33135,33136,33137,33138,33139,33140,33141,33142,33143,33144,33145,33146,33147,33149,33150,33154,33155,33156,33157,33158,33160,33161,33162,33165,33166,33167,33168,33169,33170,33172,33173,33174,33175,33176,33177,33178,33179,33180,33181,33182,33183,33184,33185,33186,33187,33189,33190,33193,33194,33196
